### This is the code to prepare the data for entailment task. The input data is the training data for mechanism generation. The output is two sets of negative and positive sentences for the entailment task. The positive ones are the sentences themselves which are selected randomly. The negative ones are the actual sentences with a small twick to completely change the sentence.

In [ ]:
import pandas as pd
import spacy
import random

train_df=pd.read_csv('generation_out_train_v8/afterpretrain_all_4_no_entit_tag_v8/19/postprocess_out.csv')
generation_df=pd.read_csv('data/v8_1/noEntity_inExp/split_data/combined_generation_neurologic_decoding.csv')
generation_nd_SEN=pd.read_csv('neurologic_decoding/seq2seq/generation_negatives/SEN/generation_combined_SEN.csv')
generation_nd_SRE=pd.read_csv('neurologic_decoding/seq2seq/generation_negatives/SRE/generation_combined_SRE.csv')
combined_input=pd.read_csv('data/v8_1/noEntity_inExp/split_data/combined_input.csv')
list(combined_input)

In [ ]:
import re
import json

def negToPos(sent):
    negative=False
    new_sent=sent
    if len(re.findall(r'\bwas not\b',sent)) > 0:
        new_sent = re.sub(r'\bwas not\b','was',sent,1)
        negative= True
        return new_sent,negative
    if len(re.findall(r'\bwere not\b',sent)) > 0:
        new_sent = re.sub(r'\bwere not\b','were',sent,1)
        negative= True
        return new_sent,negative
    if len(re.findall(r'\bcannot\b',sent)) > 0:
        new_sent = re.sub(r'\bcannot\b','can',sent,1)
        negative= True
    if len(re.findall(r'\bis not\b',sent)) > 0:
        new_sent = re.sub(r'\bis not\b','is',sent,1)
        negative= True
    if len(re.findall(r'\bisn\'t\b',sent)) > 0:
        new_sent = re.sub(r'\bisn\'t\b','is',sent,1)
        negative= True
    if len(re.findall(r'\bwasn\'t\b',sent)) > 0:
        new_sent = re.sub(r'\bwasn\'t\b','was',sent,1)
        negative= True
    if len(re.findall(r'\baren\'t\b',sent)) > 0:
        new_sent = re.sub(r'\baren\'t\b','are',sent,1)
        negative= True
    if len(re.findall(r'\bweren\'t\b',sent)) > 0:
        new_sent = re.sub(r'\bweren\'t\b','were',sent,1)
        negative= True
    return new_sent,negative

def posToNeg(sent):
    # print(sent)
    negative=False
    new_sent=sent
    if len(re.findall(r'\bwas\b',sent)) > 0:
        new_sent = re.sub(r'\bwas\b','was not',sent,1)
        negative= True
        return new_sent,negative
    if len(re.findall(r'\bwere\b',sent)) > 0:
        new_sent = re.sub(r'\bwere\b','were not',sent,1)
        negative= True
        return new_sent,negative
    if len(re.findall(r'\bdo\b',sent)) > 0:
        new_sent = re.sub(r'\bdo\b','do not',sent,1)
        negative= True
        return new_sent,negative
    
    if len(re.findall(r'\bcan\b',sent)) > 0:
        new_sent = re.sub(r'\bcan\b','cannot',sent,1)
        negative= True
        return new_sent,negative



    if len(re.findall(r'\bis\b',sent)) > 0:
        new_sent = re.sub(r'\bis\b','is not',sent,1)
        negative= True
    return new_sent,negative

    


def swap_entity_tags(sent):
    new_sent=sent.replace('<re>','<el1>').\
        replace('<er>','<le1>').\
            replace('<el>','<re>').\
                replace('<le>','<er>').\
                    replace('<el1>','<el>').\
                        replace('<le1>','<le>')
    return new_sent,True

def swap_entity_positions(sent):
    try:
        regulator=re.findall("<re>(.*?)<er>",sent)[0].strip()
        regulated=re.findall("<el>(.*?)<le>",sent)[0].strip()
        sent1=re.sub('<re> '+re.escape(regulator)+' <er>','<re1> '+regulated+ ' <er1>',sent)
        sent2=re.sub('<el> '+re.escape(regulated)+' <le>','<re> '+regulator+ ' <er>',sent1)
        sent3=re.sub('<re1> '+re.escape(regulated)+ ' <er1>','<el> '+regulated+' <le>',sent2)
    except Exception as e:
        print('error',e)
        return(sent,False)
    return(sent3,True)



def swap_entity_names(sent):
    regulator=re.findall("<re>(.*?)<er>",sent)
    regulated=re.findall("<el>(.*?)<le>",sent)
    if not regulator or not regulated:
        return sent,False
    regulator=regulator[0].strip()
    regulated=regulated[0].strip()
    sent1=re.sub('<re> '+re.escape(regulator)+' <er>','<re> '+regulated+ ' <er>',sent)
    sent2=re.sub('<el> '+re.escape(regulated)+' <le>','<el> '+regulator+ ' <le>',sent1)
    return sent2, True



# def swap_random_entity(sent):
#     regulator=re.findall("<re>(.*?)<er>",sent)[0].strip()
#     regulated=re.findall("<el>(.*?)<le>",sent)[0].strip()
#     nlp = spacy.load("en_ner_bionlp13cg_md")
#     doc=nlp(sent)
#     regulator_type=regulated_type=None
#     entity_type_text_map={}
#     for ent in doc.ents:
#         if ent.text.strip() == regulator.strip():
#             regulator_type=ent.label_
#             continue
#         if ent.text.strip() == regulated.strip():
#             regulated_type=ent.label_
#             continue    
#         if ent.label_ in entity_type_text_map:
#             entity_type_text_map[ent.label_].append(ent.text)
#         else:
#             entity_type_text_map[ent.label_]=[ent.text]
#     if not regulator_type and not regulated_type:
#         return sent,False 
#     if regulator_type in entity_type_text_map:
#         new_ent=random.choice(entity_type_text_map[regulator_type])
#         new_sent=re.sub('<re> '+regulator+' <er>','<re> temppp <er>',sent)
#         new_sent=re.sub(re.escape(new_ent),regulator,new_sent)
#         new_sent=re.sub('<re> temppp <er>','<re> '+new_ent+' <er>',new_sent)
#         return new_sent,True 
#     if regulated_type in entity_type_text_map:
#         new_ent=random.choice(entity_type_text_map[regulated_type])
#         new_sent=re.sub('<el> '+regulated+' <le>','<el> tempp <le>',sent)
#         new_sent=re.sub(re.escape(new_ent),regulated,new_sent)
#         new_sent=re.sub('<el> tempp <le>','<el> '+new_ent+ ' <le>',new_sent)
#         return new_sent,True 
#     return sent,False

def swap_random_entity_outside(sent,entity_type_map_reverse,entity_type_map):
    regulator=re.findall("<re>(.*?)<er>",sent)
    regulated=re.findall("<el>(.*?)<le>",sent)
    if not regulator or not regulated:
        return sent,False
    regulator=regulator[0].strip()
    regulated=regulated[0].strip()
    regulator_type=regulated_type=None
    if regulator.strip() in entity_type_map:
            regulator_type=entity_type_map[regulator.strip()]
    if regulated.strip() in entity_type_map:
            regulated_type=entity_type_map[regulated.strip()]
    if not regulator_type and not regulated_type:
        return sent,False 
    if regulator_type:
        possible_entities=entity_type_map_reverse[regulator_type]
        new_ent=random.choice(possible_entities)
        if new_ent.strip() != regulator.strip():
            new_sent=re.sub('<re> '+re.escape(regulator)+' <er>','<re> temppp <er>',sent)
            new_sent=re.sub(re.escape(new_ent),regulator,new_sent)
            new_sent=re.sub('<re> temppp <er>','<re> '+new_ent+' <er>',new_sent)
            return new_sent,True 
    if regulated_type:
        new_ent=random.choice(entity_type_map_reverse[regulated_type])
        if new_ent.strip() != regulated.strip():
            new_sent=re.sub('<el> '+re.escape(regulated)+' <le>','<el> tempp <le>',sent)
            new_sent=re.sub(re.escape(new_ent),regulated,new_sent)
            new_sent=re.sub('<el> tempp <le>','<el> '+new_ent+ ' <le>',new_sent)
            return new_sent,True 
    return sent,False

## select a random entity from the supporting set and replace one of the main entities with an entity
## with the same entity type
def swap_random_entity_fromFile(sent,filePath='/home/mbastan/XP/entailment',index=0):
    with open(os.path.join(filePath,'supp_set_entity_types.json')) as f:
        lines=f.readlines()
        supp_set_entity_type=json.loads(lines[index])
    with open(os.path.join(filePath,'conclusion_entity_types.json')) as f:
        lines=f.readlines()
        concl_entity_type=json.loads(lines[index])
    regulator=re.findall("<re>(.*?)<er>",sent)
    regulated=re.findall("<el>(.*?)<le>",sent)
    if not regulator or not regulated:
        return sent,False
    regulator=regulator[0].strip()
    regulated=regulated[0].strip()
    regulator_type=regulated_type=None
    entity_type_text_map={}
    if regulator.strip() in concl_entity_type:
            regulator_type=concl_entity_type[regulator.strip()]
    if regulated.strip() in concl_entity_type:
            regulated_type=concl_entity_type[regulated.strip()]
    if not regulator_type and not regulated_type:
        return sent,False 
    for ent in supp_set_entity_type:
        label=supp_set_entity_type[ent]
        if label in entity_type_text_map:
            entity_type_text_map[label].append(ent)
        else:
            entity_type_text_map[label]=[ent]

    if regulator_type in entity_type_text_map:
        new_ent=random.choice(entity_type_text_map[regulator_type])
        if new_ent.strip() != regulator.strip():
            new_sent=re.sub('<re> '+re.escape(regulator)+' <er>','<re> temppp <er>',sent)
            new_sent=re.sub(re.escape(new_ent),regulator,new_sent)
            new_sent=re.sub('<re> temppp <er>','<re> '+new_ent+' <er>',new_sent)
            return new_sent,True 
    if regulated_type in entity_type_text_map:
        new_ent=random.choice(entity_type_text_map[regulated_type])
        if new_ent.strip() != regulated.strip():
            new_sent=re.sub('<el> '+regulated+' <le>','<el> tempp <le>',sent)
            new_sent=re.sub(re.escape(new_ent),regulated,new_sent)
            new_sent=re.sub('<el> tempp <le>','<el> '+new_ent+ ' <le>',new_sent)
            return new_sent,True 
    return sent,False

## lexical polarity reverse: reverse the relational words with their antonyms
def word_replace(sent):
    negative = False
    if len(re.findall(r'\binhibits\b',sent)) > 0:
        new_sent = re.sub(r'\binhibits\b','promotes',sent,1)
        negative= True
        return new_sent,negative
    if len(re.findall(r'\bpromotes\b',sent)) > 0:
        new_sent = re.sub(r'\bpromotes\b','inhibits',sent,1)
        negative= True
        return new_sent,negative
    if len(re.findall(r'\binhibition\b',sent)) > 0:
        new_sent = re.sub(r'\binhibition\b','promotion',sent,1)
        negative= True
        return new_sent,negative
    if len(re.findall(r'\bpromotion\b',sent)) > 0:
        new_sent = re.sub(r'\bpromotion\b','inhibition',sent,1)
        negative= True
        return new_sent,negative
    if len(re.findall(r'\binhibitor\b',sent)) > 0:
        new_sent = re.sub(r'\binhibitor\b','promoter',sent,1)
        negative= True
        return new_sent,negative  
    if len(re.findall(r'\bpromoter\b',sent)) > 0:
        new_sent = re.sub(r'\bpromoter\b','inhibitor',sent,1)
        negative= True
        return new_sent,negative 

    if len(re.findall(r'\bincrease\b',sent)) > 0:
        new_sent = re.sub(r'\bincrease\b','decrease',sent,1)
        negative= True
        return new_sent,negative 

    if len(re.findall(r'\bdecrease\b',sent)) > 0:
        new_sent = re.sub(r'\bdecrease\b','increase',sent,1)
        negative= True
        return new_sent,negative 
    return sent,False     

# sswap one of the numbers frm the conclusion with another random number from the supporting set
def swapNumber(line,supp_set):
    line_nums=re.findall(r'(?:\d*\.\d+|\d+)', line)
    abs_nums=re.findall(r'(?:\d*\.\d+|\d+)', supp_set)
    if not line_nums or not abs_nums:
        return line, False
    ln_num=random.choice(line_nums)
    ab_num=random.choice(abs_nums)
    new_line=line
    if ln_num!=ab_num or '%s)'%ln_num in line: ## if the number follows by ) it means its explaining something so it's not acceptable
        # new_line=re.sub(r'\b{}[^\d.]\b'.format(ln_num),ab_num,line)
        # new_line=re.sub(r'\b[-+]?(?:{})\b'.format(ln_num),ab_num,line)
        new_line=re.sub(r'(?<!\.)\b{}\b(?!\.)'.format(ln_num),ab_num,line)
    if new_line==line:
        return(new_line,False)
    return new_line,True


# select generated examples from the pretrained models select the ones that have bleurt score lower than 0.45 and
# the relation between entities are predicted reverse from the ground truth
def generatedSamples(line,gen_row,scr_threshold=0.45):
    gen_exp = gen_row['BM_Exp']
    gen_lbl=gen_row['BM_lbl']
    gen_reg=gen_row['BM_reg']
    gen_ele=gen_row['BM_ele']
    true_lbl=gen_row['True_lbl']
    tru_reg=gen_row['True_reg']
    tru_ele=gen_row['True_ele']
    bleu_scr=gen_row['BM_scr']
    if gen_lbl == true_lbl:
        return(line,False)
    if pd.isna(gen_reg) or pd.isna(gen_ele):
        return(line,False)
    if gen_reg !=tru_reg or gen_ele!=tru_ele:
        return(line,False)
    if bleu_scr > scr_threshold:
        return(line,False)
    return(gen_exp,True)

def generatedSamples_neurologicDecoding(line,gen_row):
    gen_exp = gen_row['BM_Exp']
    gen_lbl=gen_row['BM_lbl']
    gen_reg=gen_row['BM_reg']
    gen_ele=gen_row['BM_ele']
    true_lbl=gen_row['True_lbl']
    # if gen_lbl == true_lbl:
    #     return(line,False)
    if pd.isna(gen_reg) or pd.isna(gen_ele) or pd.isna(gen_exp):
        return(line,False)
    regulators=re.findall("<re>(.*?)<er>",line)
    regulateds=re.findall("<el>(.*?)<le>",line)

    regulators_g=re.findall("<re>(.*?)<er>",gen_exp)
    regulateds_g=re.findall("<el>(.*?)<le>",gen_exp)
    if len(set(regulators_g)) >1 or len(set(regulateds_g)) >1: ## if multiple entities marked as regulator or regulatee, we don;t like it
        return(line,False)

    regulator=regulators[0].strip().lower()
    regulated=regulateds[0].strip().lower()
    regulated_g=regulateds_g[0].strip().lower()
    regulator_g=regulators_g[0].strip().lower()

    if regulator != regulated_g or regulated != regulator_g: ## if the entities are not reversed this is not what we want
        return(line,False)

    return(gen_exp,True)


def generatedSamples_nd_SEN(line,gen_row):
    
    try:
        gen_exp = gen_row['generated_nd'].iloc[0][6:-16] ## skip first <exp> token and last polarity label
        if len(set(re.findall("<el>(.*?)<le>",gen_exp))) >1 : ## multiple entities marked as element
            return(line, False)
        if len(set(re.findall("<re>(.*?)<er>",gen_exp))) >1 : ## multiple entities marked as regulator
            return(line, False)
        
        if gen_row['sat_er_and_ed'].iloc[0] < 1:
            return(line,False)
        return(gen_exp,True)
    except Exception as e:
        print('error ocurred:',e)
        return(line,False)

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
model = SentenceTransformer('michiyasunaga/BioLinkBERT-base')#'bert-base-nli-mean-tokens')
threhsold=0.94

def generatedSamples_nd_SRE(line,gen_row):
    try:
        gen_exp = gen_row['generated_nd'].iloc[0][6:-16]

        if len(set(re.findall("<el>(.*?)<le>",gen_exp))) >1 : ## multiple entities marked as element
            return(line, False)
        if len(set(re.findall("<re>(.*?)<er>",gen_exp))) >1 : ## multiple entities marked as regulator
            return(line, False)
        if gen_row['sat_er_and_ed'].iloc[0] < 1:
            return(line,False)
        sentence_embeddings = model.encode([gen_exp,line])
        if cosine_similarity([sentence_embeddings[1]],[sentence_embeddings[0]]) > 0.90: ## if two sentences are too similar skip this
                return (line,False)
        return(gen_exp,True)
    except:
        return(line,False)



In [ ]:
entity_type_map_reverse={}
entity_type_map={}
with open('entailment/supp_set_entity_types_v2.json') as f:
    lines=f.readlines()
for line in lines:
    supp_set_entity_type=json.loads(line)
    for item in supp_set_entity_type:
        if not item in entity_type_map:
            entity_type_map[item]=supp_set_entity_type[item]
        typee=supp_set_entity_type[item]
        if typee in entity_type_map_reverse:
            entity_type_map_reverse[typee].append(item)
        else:
            entity_type_map_reverse[typee]=[item]
print(len(entity_type_map_reverse))
print(len(entity_type_map))

In [ ]:
def add_row(supp_set,new_line,labels_cat,line,data_df,index,pmid,negative=0):
    ## remove extra space
    new_line=re.sub(' +', ' ', new_line)
    row = {'supp_set':supp_set.strip(),\
                'conclusion':new_line.strip(),\
                'abstract':supp_set.strip()+ ' <exp> '+new_line.strip(),\
                'label_cat':labels_cat,
                'ori_conclusion':line.strip(),
                'pmid':pmid,
                'index':index}
                
    return(data_df.append(row,ignore_index=True),negative - 1)

In [ ]:
%%time
import random
import pickle
import pandas as pd
import os
LPR_count=0 # word replace or lexical polarity reversal
negToPos_count=0
posToNeg_count=0
swap_number_count=0
generation_count=0
generation_nd_count=0
generation_nd_SRE_count=0
generation_nd_SEN_count=0
neg_constraint=0
SRE_count=0 #swap_random_entity
SEP_count=0 #swap_entity_positions
SET_count=0 #swap_entity_tags
SEN_count=0 #swap_entity_names
SREO_count=0 # swap with a random entity from outside the context
negative_sents = []
positives =0
positive_sents =[]
negative_ids =[]
positive_ids = []
negative_supp_set=[]
positive_supp_set=[]
labels= []
data_df=pd.DataFrame([])



# train_df.sample(frac=1)
# train_exp=list(train_df['True_Exp'])
# train_supp=list(train_df['Sup_Sen'])


combined_input.sample(frac=1)
train_exp=list(combined_input['conclusion'])
train_supp=list(combined_input['original_supp'])
pmids=list(combined_input['pmid'])

entity_type_map_reverse={}
entity_type_map={}
with open('entailment/supp_set_entity_types.json') as f:
    lines=f.readlines()
for line in lines:
    supp_set_entity_type=json.loads(line)
    for item in supp_set_entity_type:
        if not item in entity_type_map:
            entity_type_map[item]=supp_set_entity_type[item]
        typee=supp_set_entity_type[item]
        if typee in entity_type_map_reverse:
            entity_type_map_reverse[typee].append(item)
        else:
            entity_type_map_reverse[typee]=[item]

## entailment data versioning:
base_path="entailment_data/v10"
index=-1
null_values=0
for line,supp_set,pmid in zip(train_exp,train_supp,pmids):
    index+=1
    if index % 1000 ==0:
        print('processed %d items'%index)
    # break
    if pd.isna(line) or pd.isna(supp_set) or not re.search("<re>(.*?)<er>",line) or not re.search("<el>(.*?)<le>",line):
        null_values+=1
        continue


    if True: 
        positive_sents.append(line)
        new_line = line
        positive_supp_set.append(supp_set)
        positives += 1
        positive_ids.append(pmid)
        labels_cat = 'pos'
        data_df,_=add_row(supp_set,new_line,labels_cat,line,data_df,index,pmid)
    if True: 
        negative_supp_set.append(supp_set)
        negative_ids.append(pmid)
        negative_c =False
        negative=5
        if True: 
            c_df=generation_nd_SEN[generation_nd_SEN['pmid']==float(pmid)]
            if len(c_df) > 0:
                new_line, negative_c=generatedSamples_nd_SEN(line,c_df)

            if negative_c:
                labels_cat='generation_nd_SEN'
                negative_sents.append(new_line)
                generation_nd_SEN_count +=1
                data_df,negative=add_row(supp_set,new_line,labels_cat,line,data_df,index,pmid,negative)

        if True:
            c_df=generation_nd_SRE[generation_nd_SRE['pmid']==float(pmid)]
            if len(c_df) > 0:
                new_line, negative_c=generatedSamples_nd_SRE(line,c_df)

            if negative_c:
                labels_cat='generation_nd_SRE'
                negative_sents.append(new_line)
                generation_nd_SRE_count += 1
                data_df,negative=add_row(supp_set,new_line,labels_cat,line,data_df,index,pmid,negative)
                # repetition -=1


        if True:
            c_df=generation_df[generation_df['Sup_Sen']==supp_set]
            if len(c_df) > 0:
                new_line, negative_c=generatedSamples(line,c_df.iloc[0]) 


            if negative_c:
                labels_cat='generation'
                negative_sents.append(new_line)
                generation_count +=1
                data_df,negative=add_row(supp_set,new_line,labels_cat,line,data_df,index,pmid,negative)
                # repetition -=1

        if True:#if  repetition>0 and negative>0 and random.random()<1:
            try:
                new_line,negative_c=swap_random_entity_fromFile(line,index=index)
            except Exception as e:
                # print('error darim',str(e))
                pass
            if negative_c:
                labels_cat='SRE' # swap random entity
                negative_sents.append(new_line)
                SRE_count +=1
                data_df,negative=add_row(supp_set,new_line,labels_cat,line,data_df,index,pmid,negative)


        if True:
            new_line,negative_c=swap_entity_positions(line)
            if negative_c:
                labels_cat='SEP' # swap entity positions
                negative_sents.append(new_line)
                SEP_count +=1
                data_df,negative=add_row(supp_set,new_line,labels_cat,line,data_df,index,pmid,negative)


        if True:
            new_line,negative_c=swap_entity_tags(line)
            if negative_c:
                labels_cat='SET' # swap entity tags
                negative_sents.append(new_line)
                SET_count +=1
                data_df,negative=add_row(supp_set,new_line,labels_cat,line,data_df,index,pmid,negative)


        if True:
            c_df=generation_df[generation_df['pmid']==float(pmid)]
            if len(c_df) > 0:
                new_line,negative_c=generatedSamples_neurologicDecoding(line,c_df.iloc[0])
                if negative_c:
                    labels_cat='generation_nd' # swap entity names
                    negative_sents.append(new_line)
                    generation_nd_count +=1
                    data_df,negative=add_row(supp_set,new_line,labels_cat,line,data_df,index,pmid,negative)
                    

        if True:
            new_line,negative_c=swap_random_entity_outside(line,entity_type_map_reverse,entity_type_map)
            if negative_c:
                labels_cat='SREO' # swap random entity from outside the txt
                negative_sents.append(new_line)
                SREO_count +=1
                data_df, negative=add_row(supp_set,new_line,labels_cat,line,data_df,index,pmid,negative)



        if True:
            new_line, negative_c=swapNumber(line,supp_set)

            if negative_c:
                labels_cat='swap_number'
                negative_sents.append(new_line)
                swap_number_count +=1
                data_df, negative=add_row(supp_set,new_line,labels_cat,line,data_df,index,pmid,negative)



        if True:
            new_line,negative_c=word_replace(line)
            if negative_c:
                labels_cat='LPR'
                negative_sents.append(new_line)
                LPR_count += 1 
                data_df,negative=add_row(supp_set,new_line,labels_cat,line,data_df,index,pmid,negative)



        if True:
            new_line,negative_c=posToNeg(line)
            if negative_c:
                labels_cat='posToNeg'
                negative_sents.append(new_line)
                posToNeg_count += 1 
                data_df,negative=add_row(supp_set,new_line,labels_cat,line,data_df,index,pmid,negative)


        if True:
                new_line,negative_c=negToPos(line)
                if negative_c:
                    labels_cat='negToPos'
                    negative_sents.append(new_line)
                    negToPos_count += 1
                    data_df,negative=add_row(supp_set,new_line,labels_cat,line,data_df,index,pmid,negative)



        if True:
            new_line,negative_c=swap_entity_names(line)
            if negative_c:
                labels_cat='SEN' # swap entity names
                negative_sents.append(new_line)
                SEN_count +=1
                data_df,negative=add_row(supp_set,new_line,labels_cat,line,data_df,index,pmid,negative)

    if index % 1000 ==0:
        data_df.to_csv(os.path.join(base_path,'entailment_data.csv'),index=False)
        with open(os.path.join(base_path,'positive_conclusion.txt'),'w') as f:
            for item in positive_sents:
                f.write(item)
                f.write('\n')
        with open(os.path.join(base_path,'negative_conclusion.txt'),'w') as f:
            for item in negative_sents:
                f.write(item)
                f.write('\n')

        with open(os.path.join(base_path,'positive_suppset.txt'),'w') as f:
            for item in positive_supp_set:
                f.write(item)
                f.write('\n')
        with open(os.path.join(base_path,'negative_suppset.txt'),'w') as f:
            for item in negative_supp_set:
                f.write(item)
                f.write('\n')

        with open(os.path.join(base_path,'negative_ids.pk'),'wb') as f:
            pickle.dump(negative_ids,f)
        with open(os.path.join(base_path,'positive_ids.pk'),'wb') as f:
            pickle.dump(positive_ids,f)

        with open(os.path.join(base_path,'labels_cat.pk'),'wb') as f:
            pickle.dump(labels,f)

print('number of null values:', null_values)

In [ ]:
print('positives: %d, LPR_count: %d, generation_count: %d ,generation_nd_count: %d, generation_nd_SRE_count: %d, generation_nd_SEN_count: %d ,SET_count: %d ,SEN_count: %d \
,SEP_count: %d ,SRE_count: %d ,SREO_count: %d , swap_number_count: %d, posToNeg: %d, negToPos: %d'%\
        (positives, LPR_count, generation_count,generation_nd_count, generation_nd_SRE_count, generation_nd_SEN_count, SET_count,SEN_count,\
            SEP_count,SRE_count, SREO_count, swap_number_count, posToNeg_count, negToPos_count))
len(data_df)

In [ ]:
data_df.to_csv(os.path.join(base_path,'entailment_data.csv'),index=False)
with open(os.path.join(base_path,'positive_conclusion.txt'),'w') as f:
    for item in positive_sents:
        f.write(item)
        f.write('\n')
with open(os.path.join(base_path,'negative_conclusion.txt'),'w') as f:
    for item in negative_sents:
        f.write(item)
        f.write('\n')

with open(os.path.join(base_path,'positive_suppset.txt'),'w') as f:
    for item in positive_supp_set:
        f.write(item)
        f.write('\n')
with open(os.path.join(base_path,'negative_suppset.txt'),'w') as f:
    for item in negative_supp_set:
        f.write(item)
        f.write('\n')

with open(os.path.join(base_path,'negative_ids.pk'),'wb') as f:
    pickle.dump(negative_ids,f)
with open(os.path.join(base_path,'positive_ids.pk'),'wb') as f:
    pickle.dump(positive_ids,f)

with open(os.path.join(base_path,'labels_cat.pk'),'wb') as f:
    pickle.dump(labels,f)

In [ ]:
import seaborn as sns
percentage = lambda i: len(i) / float(len(count_list)) * 100

per_list=[]
for item in set(count_list):
    per_list.append(count_list.count(item)/float(len(count_list)) * 100)

ax = sns.barplot(x=count_list, y=count_list,  estimator=percentage, color='b',edgecolor='black')

ax.set(ylabel="Percent")
ax.set(xlabel="Number of Possible Perturbations")
ax.set(title='Possible Perturbations in Dev Set')
patches = ax.patches
for i in range(len(patches)):
   x = patches[i].get_x() + patches[i].get_width()/2
   y = patches[i].get_height()+.05
   ax.annotate('{:.1f}%'.format(per_list[i]), (x, y), ha='center')
fig=ax.get_figure()
fig.savefig('data_dist.pdf')
plt.show()